In [1]:
from bs4 import BeautifulSoup  # Importa BeautifulSoup, utilizado para parsear documentos HTML (no se usa en este script).
import pandas as pd  # Importa pandas, una biblioteca para la manipulación y análisis de datos.
import requests  # Importa requests para realizar peticiones HTTP.
import re  # Importa re, una biblioteca para trabajar con expresiones regulares (no se usa en este script).

pd.set_option('display.max_columns', None)  # Configura pandas para mostrar todas las columnas de los DataFrames sin truncar.

url = "https://moviesdatabase.p.rapidapi.com/titles"  # URL de la API para solicitar datos de películas.

headers = {  # Cabeceras necesarias para autenticar las peticiones a la API.
    "X-RapidAPI-Key": "431e779057msh1f0c87c2937a329p125efcjsn45064e4b4925",
    "X-RapidAPI-Host": "moviesdatabase.p.rapidapi.com"
}

querystring = {  # Parámetros para la consulta API, filtrando películas por género, año y tipo.
    "startYear": "2010",
    "titleType": "movie",
    "endYear": "2024"
}

def llamar_pelis(url, headers, querystring):  # Define una función para realizar peticiones a la API y manejar paginación.
    datos_paginas = []  # Lista para acumular los datos de todas las páginas.
    base_url = "https://moviesdatabase.p.rapidapi.com"  # URL base para las peticiones.
    while url:  # Bucle que se ejecuta mientras haya una URL para procesar.
        if not url.startswith('http'):  # Añade la base URL si la URL proporcionada no es absoluta.
            url = base_url + url
        response = requests.get(url, headers=headers, params=querystring)  # Realiza la petición HTTP.
        if response.status_code == 200:  # Chequea si la petición fue exitosa.
            json_pelis = response.json()  # Obtiene el JSON de la respuesta.
            datos_paginas.extend(json_pelis['results'])  # Añade los resultados a la lista.
            url = json_pelis.get('next')  # Actualiza la URL para la siguiente página, si existe.
            querystring = {}  # Limpia el querystring porque la URL de 'next' ya incluye los parámetros necesarios.
        else:
            print("Error en la solicitud:", response.status_code, response.reason)  # Imprime un mensaje de error si la solicitud falla.
            return None
    return {"results": datos_paginas}  # Devuelve los datos acumulados en formato de diccionario.

# Llamada completa con paginación
datos_limpios = llamar_pelis(url, headers, querystring)  # Realiza la llamada a la función para obtener todos los datos de la API.
if datos_limpios:
    print(datos_limpios)  # Imprime los datos obtenidos para verificar.
else:
    print("Opsss, parece que no se han recibido datos")  # Mensaje de error si no se obtuvieron datos.

def limpiar_json(datos_json):  # Función para procesar los datos JSON y convertirlos en un DataFrame.
    peliculas = []  # Lista para almacenar datos de cada película.
    for pelicula in datos_json['results']:  # Itera sobre cada película en los resultados.
        tipo = pelicula['titleType']['text']  # Extrae el tipo de título.
        nombre = pelicula['titleText']['text']  # Extrae el nombre de la película.
        year = pelicula['releaseYear']['year']  # Extrae el año de lanzamiento.
        mes = pelicula['releaseDate']['month'] if 'releaseDate' in pelicula and pelicula['releaseDate'] is not None else 'Desconocido'  # Extrae el mes de lanzamiento, si está disponible.
        id_pelicula = pelicula['id']  # Extrae el ID de la película.

        # Agrega los datos de la película a la lista.
        peliculas.append({
            'Tipo': tipo,
            'Nombre': nombre,
            'Año de Estreno': year,
            'Mes de Estreno': mes,
            'ID': id_pelicula
        })

    dataframe_peliculas = pd.DataFrame(peliculas)  # Convierte la lista de películas en un DataFrame.
    
    return dataframe_peliculas  # Devuelve el DataFrame.

datos_limpios = llamar_pelis(url, headers, querystring)  # Vuelve a llamar a la función para asegurar que se obtienen los datos.
df = limpiar_json(datos_limpios)  # Procesa los datos JSON y los convierte en DataFrame.

# Imprime el DataFrame para verificar los datos
print(df)

# Guarda el DataFrame en un archivo CSV
df.to_csv("Todas las películas.csv", index=False)  # Guarda el DataFrame en un archivo CSV sin incluir el índice.


{'results': [{'_id': '61e58ba58a5a6a599f43b4c7', 'id': 'tt10080198', 'primaryImage': {'id': 'rm1939411713', 'width': 2025, 'height': 3000, 'url': 'https://m.media-amazon.com/images/M/MV5BNWVhMTZkYTItMDI4ZC00ODBhLTg3MjktN2NjMTFmMzllM2ZjXkEyXkFqcGdeQXVyNjA3NDEyMTE@._V1_.jpg', 'caption': {'plainText': "Kurt Oberhaus, Michael Joseph Pierce, Mohamed Hakeemshady, Bella Dontine, Justin Titus, and Robert Artz in A Lion's Game (2024)", '__typename': 'Markdown'}, '__typename': 'Image'}, 'titleType': {'displayableProperty': {'value': {'plainText': '', '__typename': 'Markdown'}, '__typename': 'DisplayableTitleTypeProperty'}, 'text': 'Movie', 'id': 'movie', 'isSeries': False, 'isEpisode': False, 'categories': [{'value': 'movie', '__typename': 'TitleTypeCategory'}], 'canHaveEpisodes': False, '__typename': 'TitleType'}, 'titleText': {'text': "A Lion's Game", '__typename': 'TitleText'}, 'originalTitleText': {'text': "A Lion's Game", '__typename': 'TitleText'}, 'releaseYear': {'year': 2024, 'endYear': 